In [ ]:
# Attempt to rollup most of NSE programs into ONE

# STATUS: Incomplete (Trial with index options)
# Run-time: Unknown

# Dependencies:
#  - List of indexes and equities
#  - underlying's ohlc history for standard deviation and Black Scholes

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=5) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=5) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=5) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=5) # kavi IBG paper-trade

In [ ]:
%%time
#***      Imports    *****
import pandas as pd
import numpy as np
from itertools import product

#***    Paths and Variables    ***
datapath = r'./zdata/'
market = 'NSE'
deletesym = ['USDINR']   #ibSymbols to be deleted

#***   Error catching for list comprehension ***
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan

#***    Get scrips from IBKR   ***
df_ib = pd.DataFrame([]) # Initialize

# from IBKR's symbols page
url = "https://www.interactivebrokers.com.hk/en/index.php?"
url = url+"f=2222&exch=nse&showcategories=OPTGRP&page="

for i in range(4):
    df_ib = df_ib.append(df_ib.append(pd.read_html(url+str(i), header=0)[2]))

# Exit program if dataframe is empty
if df_ib.empty:
    print('Interactive Brokers dataframe is empty!!!')
    sys.exit(0)
    
df_ib = df_ib.drop_duplicates().reset_index(drop=True)

# Qualify Stock contracts
stocks = [Stock(symbol=s, exchange=market) for s in df_ib['IB Symbol']]
nse_qs = ib.qualifyContracts(*stocks)

df_qs = util.df(nse_qs) # data frame of qualified stocks

# Get a df of indexes
df_ix = df_ib[~df_ib['IB Symbol'].isin(df_qs.symbol)]

# Qualify the indexes
indexes = [Index(symbol=s, exchange='NSE') for s in df_ix['IB Symbol']]
nse_qi = ib.qualifyContracts(*indexes)

df_qi = util.df(nse_qi) # dataframe of qualified indexes

symbols = nse_qi + nse_qs

# symbols = nse_qi # DATA LIMITER!!!

nse_q = [n for n in symbols if n.symbol not in deletesym]   # remove USDINR

df_q = util.df(nse_q)  # dataframe of qualified stocks + indexes

df_q['contract'] = pd.Series(nse_q)

In [ ]:
#*** Get the volatilities
def get_maxvol(contract):
    '''Gets maximum volatailty
       Arg: (contract) as object
       Returns: maxvol as float - maximum of 12 month's HV and IV '''
    bars1 = ib.reqHistoricalData(contract, endDateTime='', durationStr='12 M',
            barSizeSetting='1 day', whatToShow='OPTION_IMPLIED_VOLATILITY', useRTH=True)
    bars2 = ib.reqHistoricalData(contract, endDateTime='', durationStr='12 M',
            barSizeSetting='1 day', whatToShow='HISTORICAL_VOLATILITY', useRTH=True)
    maxvol = max([b.close for b in bars1]+[b.close for b in bars2])
    return maxvol

df_q['maxvol'] = df_q.contract.apply(get_maxvol)

In [ ]:
#***  Get the underlying's price
def get_price(contract):
    '''Gets the price of the contract
    Arg: (contract) as object
    Returns: close priceas float'''
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='60 S',
            barSizeSetting='1 secs',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1,
            keepUpToDate=True)[-1:]

    close = [b.close for b in bars]
    
    return close[0]

In [ ]:
# df_q.contract.apply(get_price)

In [ ]:
#***   Error catching for list comprehension ***
import numpy as np
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
df_q['closeprice'] = [catch(lambda: getprice(c)) for c in df_q.contract]

In [ ]:
get_price(df_q.contract[3])

In [ ]:
df_q['closeprice'] = [get_price(c) for c in df_q.contract]

In [ ]:
# ***** Get list with Mlots + Margin% from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

df_paisa1 = df_paisa[['Symbol', 'Mlot', 'TotMgn%']].sort_values('Symbol').reset_index(drop=True)

# Rename Symbol and Margin fields
df_paisa1 = df_paisa1.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa1 = df_paisa1.apply(pd.to_numeric, errors='ignore')
df_paisa1.marginpct = df_paisa1.marginpct.div(100)

In [ ]:
#***    make option chains and qualify   ***
chains = {s: ib.reqSecDefOptParams(s.symbol, '', s.secType, s.conId) for s in symbols}
chains

df_und = util.df(chains) # dataframe of underlying chains

tup_chains = [([k.symbol], tuple(i.expirations), tuple(i.strikes), ['P']) 
 for k, v in chains.items() 
 for i in v]

# create options without USDINR scrip
options = [Option(p[0], p[1], p[2], p[3], 'NSE') 
           for tup in tup_chains
           for p in product(*tup)]

# qualify the options
nse_qo = [t for i in range(0, len(options), 25) for t in ib.qualifyContracts(*options[i: i+25])]

df_qo = util.df(nse_qo) # dataframe for qualified options

In [ ]:
contract = nse_qo[0]
order = Order(action='SELL', orderType='MKT', totalQuantity=20)
ib.whatIfOrder(contract, order)

In [ ]:
ib.reqSecDefOptParams(futFopExchange='NSE', underlyingConId=337009725, underlyingSecType='IND', underlyingSymbol = 'BANKNIFTY')

In [ ]:
ib.disconnect()

# To-do
* something is wrong in the qual_options. Fix it.
* integration with standard deviation
* remove unnecessary options (Put)
* integration with black scholes
* integration with lot and margin
* generate orders
* place orders (what-if) to check margins (duplicate)
* check latest prices
* screen most valueable RoM and PoP
* place orders